# Logistische Regression auf dem Titanic Datensatz

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Hole Daten

Der Datensatz stammt von [Kaggle](https://www.kaggle.com/sureshbhusare/titanic-dataset-from-kaggle).

Er beinhaltet Daten zu den über 1.300 Passagieren der tragischen Jungfernfahrt.

Die Daten der über 900 Mann starken Besatzung sind nicht enthalten.

Die Spalten haben folgende Bedeutung:

Variable	|Definition	|Key
:---|:---|---
Survival	|Survival - Label	|0 = No, 1 = Yes
Pclass	|Ticket class	|1 = 1st, 2 = 2nd, 3 = 3rd
Sex	|Sex	| male, female
Age	|Age in years|	
SibSp	|# of siblings / spouses aboard the Titanic	|
Parch	| # of parents / children aboard the Titanic	|
Ticket	|Ticket number	|
Fare	|Passenger fare	|
Cabin	|Cabin number	|
Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton
Name|Name des Passagiers |


In [ ]:
train = pd.read_csv('./data/titanic/train.csv')
test = pd.read_csv('./data/titanic/test.csv')

## Ein erster Blick auf die Daten

`shape` sagt uns die Dimensionen des Datensatzes: Wieviele Datensätze (Zeilen) und wieviele Features (Spalten) haben wir?

In [ ]:
print(train.shape)

Mit `head` oder `tail` können wir uns die ersten bzw. letzten paar Datensätze anschauen:

In [ ]:
train.head()

In [ ]:
train.tail()

`info` gibt uns ein paar Informationen zu den Datentypen der Spalten:

In [ ]:
train.info();

`describe` liefert uns einige Kennzahlen zur statistischen Verteilung der Daten:

In [ ]:
train.describe(exclude='O')

Wir konnten aus `train.info()` schon ablesen, dass zu "Age", "Embarked" und "Cabin" weniger non-null Werte als 891 angezeigt werden. Zu diesen Null-Werten müssen wir noch überlegen, wie wir damit umgehen.

Hier können wir uns ein Bild davon machen, wo die Null-Werte auftreten:

In [ ]:
sns.heatmap(data = train.isnull(), yticklabels=False, cbar=False, cmap='viridis');

## Exploratory Data Analysis

... wie sehen unsere Daten eigentlich aus?



Plotten wir mal eine Verteilung:

In [ ]:

f, axes = plt.subplots(2, 4, figsize=(20, 7), sharex=False)

showColumns = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

r = 0
c = 0
for col in showColumns:
    sns.countplot(train[col], ax=axes[r, c])
    c = c + 1
    if (c > 3):
        r = 1
        c = 0


"Fare" sieht seltsam aus, schauen wir uns mal näher an:

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(20, 7), sharex=False)
sns.distplot(train['Fare'], ax=axes[0])
sns.countplot(train['Fare'], ax=axes[1]);

### Correlation

In der Statistik misst der Korrelationskoeffizient r die Stärke und Richtung einer linearen Beziehung zwischen zwei Variablen.

Ab einem Absolutwert größer 0.5 sollte man die Korrelation betrachten.

In [ ]:
sns.heatmap(train.corr(), annot=True);

## Daten Aufbereitung

### Fehlende Werte: Cabin

Die Spalte "Cabin" ist nur zu einem geringen Teil gefüllt und es scheint nicht plausibel, dass aus der Kabinenbezeichnung 
ein Einfluss auf die Überlebenswahrscheinlichkeit ausgeht, daher droppen wir diese Spalte:

In [ ]:
train.drop(['Cabin'], axis=1, inplace=True)
train.info();

## Fehlende Werte: Age

Die Spalte "Age" hat einige fehlende Einträge. Wir haben die Alternativen entweder die Spalte auch zu droppen, 
dann verlieren wir allerdings einige Information, oder die fehlenden Werte aufzufüllen.

Ein Ansatz dazu, ist zu schauen, wo die höchste Korrelation zu "Age" besteht und dies zum Auffüllen zu verwenden:
"Pclass" ist am höchsten korreliert mit "Age".

In [ ]:
age_group = train.groupby("Pclass")["Age"]
print(age_group.median())

Der Altersmedian unterscheided sich zwischen den Klassen signifikant, also setzen wir diesen für die fehlenden Werte ein:

In [ ]:
train.loc[train.Age.isnull(), 'Age'] = train.groupby("Pclass").Age.transform('median')
print(train["Age"].isnull().sum())

Unsere Altersverteilung sieht jetzt so aus:

In [ ]:
plt.figure(figsize = (16, 8))

sns.distplot(train["Age"])
plt.title("Age Histogram")
plt.xlabel("Age")
plt.show()

### Fehlende Werte: Embarked

Für die zwei fehlenden Ausgangshäfen nehmen wir einfach den häufigsten Wert an - "S":

In [ ]:
train["Embarked"] = train["Embarked"].fillna('S')
train.isnull().sum()

... keine fehlenden Werte mehr.

### Spalten droppen, die wir nicht weiter betrachten

In [ ]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Fare'], axis = 1, inplace = True)
train.head()

### One-hot Encoding "Sex" und "Embarked"

Pandas hat eine sehr nützliche Funktion, die kategorische Variablen One-Hot encoded und den Dataframe gleich entsprechend umwandelt - diese heißt `get_dummies`.

Der Name rührt daher, dass mit dem On-Hot Encoding neue Spalten (= neue Variablen) entstehen, die "Dummy-Variablen" genannt
werden - da sie in gewissem Sinne ja keine "echten" Variablen sind.

In [ ]:
train_onehot = pd.get_dummies(train, drop_first=True)
train_onehot

## Training vorbereiten

`train` enthält ja sowohl unsere Features als auch das Label `Survived`. Dies teilen wir jetzt auf in X und y:

In [ ]:
y = train_onehot['Survived']
X = train_onehot.drop('Survived', axis=1)
X, y

Schließlich splitten wir unsere Daten noch in ein Training- und ein Test-Set im Verhältnis 80:20.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Logistische Regression

Trainiere das Modell

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

... und bewerte das Ergebnis ...

**Confusion Matrix:** Die Confusion-Matrix $C$ ist definiert durch: $C_{i,j}$ ist die Anzahl der Beobachtungen der wahren Gruppe $i$, die als zur Gruppe $j$ vorhergesagt werden.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print("\t'false'\t'true'")
print("false\t  {}\t  {}".format(tn, fp))
print("true\t  {}\t  {}".format(fn, tp))

### Kennzahlen zur Bewertung

**Precision** ist das Verhältnis $\frac{tp}{tp + fp}$, wobei $tp$ die Anzahl der echten positiven und $fp$ die Anzahl der falschen positiven Werte ist. Intuitiv ist Precision die Fähigkeit des Klassifikators, eine negative Probe nicht als positiv zu kennzeichnen.

**Recall** ist das Verhältnis $\frac{tp}{tp + fn}$, wobei $fn$ die Anzahl der falschen Negativen ist. Intuitiv ist Recall die Fähigkeit des Klassifikators, alle positiven Proben zu finden.

Der **F-Beta-Score** kann als ein gewichteter harmonischer Mittelwert von Precision und Recall interpretiert werden, wobei ein F-Beta-Score seinen besten Wert bei 1 und den schlechtesten Wert bei 0 erreicht.

$f1=2\times\frac{Precision \times Recall}{Precision + Recall}$

Der F-Beta-Score gewichtet um den Faktor Beta Recall mehr als die Precision. Beta == 1,0 bedeutet, dass Recall und Precision gleich wichtig sind.

**Support** ist die jeweilige Anzahl der Vorkommnisse der wahren Labels.

In [ ]:
print(classification_report(y_test, predictions, target_names=['Nicht überlebt', 'Überlebt']))

### Interpretation des Modells

Schauen wir uns Intercept und Koeffizienten an

In [ ]:
print('Intercept/Bias: {}'.format(model.intercept_))

coef_dict = sorted(list(zip(X.columns.tolist(), model.coef_.ravel())), key=lambda tup: tup[1])
for tup in coef_dict:
    print(tup)

Das Geschlecht ist mit Abstand der am stärksten eingehende Faktor, gefolgt von der Klasse.

Da wir die Features nicht skaliert haben, stimmt die Reihenfolge bzgl SibSp, Parch und insbesondere Age so allerdings nicht.

In [ ]:
X['Age'].mean()

d.h. Age geht eigentlich mit einem fast 30-fachen Gewicht gegenüber den 0/1-Features ein und steht damit eher an dritter Stelle.

SibSp und Parch könnte man vielleicht als Familie zusammenfassen (Familiengröße = 1 + SibSp + Parch) und dann trainieren, oder eventuell die Familiengröße auch One-Hot Encoden.

Das geht dann in Richtung "Feature Engineering". Ein interessanter Artikel dazu im Kontext des Titanic Datasets 
ist [hier](https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9) zu finden.


## Anhang: LogisticRegression with Cross Validation

Abwandlung der "normalen" LogisticRegression mit automatischer Bestimmung der Hyperparameter.
(Hier Regularisierung)

Leicht bessere Ergebnisse.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV()
model_cv.fit(X_train, y_train)

predictions_cv = model_cv.predict(X_test)
tn_cv, fp_cv, fn_cv, tp_cv = confusion_matrix(y_test, predictions_cv).ravel()
print("\t'false'\t'true'")
print("false\t  {}\t  {}".format(tn_cv, fp_cv))
print("true\t  {}\t  {}".format(fn_cv, tp_cv))

print(classification_report(y_test, predictions_cv, target_names=['Nicht überlebt', 'Überlebt']))